# Calcul des variables explicatives : Les facteurs de production

( relatif à la partie ***3.3 - Les variables explicatives*** du rapport) 

In [1]:
# importation des packages
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\Vinciane\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Importations des bases :
path_ISA = r'D:/Données/ENSAE/2A/Stat_App/ISA.csv'
ISA=pd.read_csv(path_ISA ,sep=',', encoding='latin-1')

# Pour la variable explicative, les bases qui nous intéressent sont les bases Q20,Q21,Q22 et Q23
path_Q20 = r'D:/Données/ENSAE/2A/Stat_App/SAE_2019/SAE 2019 Bases statistiques - formats SAS-CSV/Bases statistiques/Bases CSV/Q20_2019r.csv'
Q20=pd.read_csv(path_Q20 ,sep=';', encoding='latin-1')
path_Q22 = r'D:/Données/ENSAE/2A/Stat_App/SAE_2019/SAE 2019 Bases statistiques - formats SAS-CSV/Bases statistiques/Bases CSV/Q22_2019r.csv'
Q22=pd.read_csv(path_Q22 ,sep=';', encoding='latin-1')
path_Q23 = r'D:/Données/ENSAE/2A/Stat_App/SAE_2019/SAE 2019 Bases statistiques - formats SAS-CSV/Bases statistiques/Bases CSV/Q23_2019r.csv'
Q23=pd.read_csv(path_Q23 ,sep=';', encoding='latin-1')

C:\Users\Vinciane\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1) Les établissements hors France métropolitaine [A ajouter dans notebook de Louise]

A partir de maintenant, on **supprime les établissements hors France métropolitaine**. On pourrait les étudier à part dans un second temps.

Arguments : 
- différences multiples avec les hopitaux en France métropolitaine
- certaines jointures impossibles
- moins de réponses à l'enquête SAE (notamment sur la base Q20)

In [3]:
ISA = ISA[~ISA['Numero_finess'].astype(str).str.startswith('97')] #Suppression des établissements dans des départements d'Outre-Mer
ISA = ISA[~ISA['Numero_finess'].astype(str).str.startswith('98')] 

In [4]:
ISA

,Numero_finess,Points_ISA,stjr
0,010008407,165054.798333,1
1,010009132,5715.170000,1
2,010780054,787929.068333,1
3,010780062,188174.950000,1
4,010780096,90756.543333,1
...,...,...,...
1173,950300244,127339.916667,2
1174,950300277,140779.191667,2
1175,950300301,6779.931667,2
1176,950300350,11099.390000,2


In [5]:
ISA['Numero_finess'].nunique()

1164

La base ISA compte désormais 1164 établissements, tous situés en France Métropolitaine.

## 1)  Effectuer la jonction entre la table ISA et les tables Q20,Q21,Q22 et Q23 

Tout d'abord, nous souhaitons ne garder que les établissements qui nous intéressent, ie ceux qui sont également dans la base ISA. Pour avoir un même "Numero_finess" sur toutes nos bases, on reprend le travail effectué pour la variable d'intérêt.

### A) Jointure entre la base Q20 (personnel médical) et la base ISA

La base ISA comporte 1173 établissements en France métropolitaine. Certains établissements de la base ISA sont représentés dans les bases Q20, Q22 et Q23 de la SAE par le numéro finess (fi) et d'autres par le numero finess de l'entité juridique(fi_ej).

Test méthode B

In [6]:
Q20_short_fi = Q20[["FI","PERSO","AN"]]
Q20_short_fi.groupby(by=["FI", "PERSO"]).count().sort_values(by="AN")
Q20_group_fi = Q20_short_fi.groupby(by=["FI", "PERSO"]).count().sort_values(by="AN")
Q20_group_fi = Q20_group_fi.rename(columns = {'AN': 'Nb_doublons_fi'})

Q20_group_fi = Q20_group_fi.reset_index()

#On réindexe, pour avoir une colonne avec les identifiants de chaque intervalle, 
#qu'on appelle id_int pour identifiant intervalle
Q20_group_fi['id_fi_perso'] = Q20_group_fi.index
Q20_group_fi.sort_values(by="Nb_doublons_fi")
Q20_group_fi

,FI,PERSO,Nb_doublons_fi,id_fi_perso
0,970100012,M2130,1,0
1,680000874,M1010,1,1
2,680000833,M9999,1,2
3,680000833,M3040,1,3
4,680000833,M1320,1,4
...,...,...,...,...
33726,340022979,M1160,1,33726
33727,340022979,M1150,1,33727
33728,340022979,M1140,1,33728
33729,340022979,M1070,1,33729


In [7]:
Q20_fi=pd.merge(ISA, Q20, left_on = ['Numero_finess'], right_on = ['FI'],how='inner')
Q20_fi

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFSALPLH,EFFSALPLF,...,ETPSALF,ETP_PU,ETP_PH,ETP_AS,ETP_HU,ETP_AT,ETP_AU,EFFSAL,ETPSAL,EFFLIB
0,060000528,356892.030000,3,Q20,2019,060000528,060780962,M1010,1.0,NaN,...,1.96,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.44,NaN
1,060000528,356892.030000,3,Q20,2019,060000528,060780962,M1020,1.0,2.0,...,2.90,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.90,NaN
2,060000528,356892.030000,3,Q20,2019,060000528,060780962,M1030,5.0,3.0,...,3.76,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9.32,NaN
3,060000528,356892.030000,3,Q20,2019,060000528,060780962,M1040,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.19,NaN
4,060000528,356892.030000,3,Q20,2019,060000528,060780962,M1050,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11016,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3011,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
11017,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
11018,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3040,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.00,NaN
11019,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3050,1.0,1.0,...,1.00,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.00,NaN


In [8]:
print("fi : ",Q20_group_fi["id_fi_perso"].size,'\n',"fi uniques :",Q20_group_fi["id_fi_perso"].nunique())
# On remarque qu'il n'y a pas de doublons pour FI

fi :  33731 
 fi uniques : 33731


In [9]:
Q20_short_fiej= Q20[["FI_EJ","PERSO","AN"]]

In [10]:
Q20_short_fiej.groupby(by=["FI_EJ", "PERSO"]).count().sort_values(by="AN") 
# On remarque tout de suite que certaines lignes sont des doublons.

AN
FI_EJ     PERSO    
750826307 M1010   1
660000282 M1340   1
          M1200   1
          M1160   1
          M1150   1
...              ..
350000626 M9999  39
920030269 M1320  43
          M3050  61
          M1010  67
          M9999  76

[26546 rows x 1 columns]

In [11]:
Q20_group_fiej = Q20_short_fiej.groupby(by=["FI_EJ", "PERSO"]).count().sort_values(by="AN")
Q20_group_fiej = Q20_group_fiej.rename(columns = {'AN': 'Nb_doublons_fiej'})

#On a encore engine_serial_number et Interpolate_WW_rank en index, on les enlève
Q20_group_fiej = Q20_group_fiej.reset_index()

#On réindexe, pour avoir une colonne avec les identifiants de chaque intervalle, 
#qu'on appelle id_int pour identifiant intervalle
Q20_group_fiej['id_fiej_perso'] = Q20_group_fiej.index
Q20_group_fiej.sort_values(by="Nb_doublons_fiej")
Q20_group_fiej


,FI_EJ,PERSO,Nb_doublons_fiej,id_fiej_perso
0,750826307,M1010,1,0
1,660000282,M1340,1,1
2,660000282,M1200,1,2
3,660000282,M1160,1,3
4,660000282,M1150,1,4
...,...,...,...,...
26541,350000626,M9999,39,26541
26542,920030269,M1320,43,26542
26543,920030269,M3050,61,26543
26544,920030269,M1010,67,26544


In [12]:
print("On a", Q20_group_fiej[(Q20_group_fiej.Nb_doublons_fiej > 1)].shape[0], "cas où il y plusieurs lignes pour un même type de métier d'un même hôpital")

On a 3252 cas où il y plusieurs lignes pour un même type de métier d'un même hôpital


In [13]:
Q20_id = pd.merge(Q20, Q20_group_fiej, left_on = ["FI_EJ", "PERSO"], right_on = ["FI_EJ", "PERSO"], how="inner")
Q20_id

,BOR,AN,FI,FI_EJ,PERSO,EFFSALPLH,EFFSALPLF,EFFSALPAH,EFFSALPAF,EFFLIBPLH,...,ETP_PH,ETP_AS,ETP_HU,ETP_AT,ETP_AU,EFFSAL,ETPSAL,EFFLIB,Nb_doublons_fiej,id_fiej_perso
0,Q20,2019,010000024,010780054,M1010,5.0,9.0,3.0,9.0,NaN,...,13.30,NaN,NaN,1.56,3.88,26.0,18.74,7.0,1,16796
1,Q20,2019,010000024,010780054,M1020,1.0,2.0,NaN,NaN,NaN,...,2.33,1.00,NaN,NaN,NaN,3.0,3.33,NaN,1,16795
2,Q20,2019,010000024,010780054,M1030,8.0,3.0,1.0,4.0,NaN,...,10.10,0.66,NaN,1.00,2.40,16.0,14.16,NaN,2,24421
3,Q20,2019,010009314,010780054,M1030,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.13,NaN,NaN,0.13,NaN,2,24421
4,Q20,2019,010000024,010780054,M1031,5.0,1.0,1.0,1.0,NaN,...,5.25,0.58,NaN,0.10,0.80,8.0,6.73,NaN,1,16794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33571,Q20,2019,980500011,980500003,M3060,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.01,1.0,1.01,NaN,1,16628
33572,Q20,2019,980500011,980500003,M9999,110.0,74.0,23.0,19.0,NaN,...,99.18,1.35,NaN,13.21,111.24,226.0,224.98,NaN,1,16627
33573,Q20,2019,980500763,970407250,M1110,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,2.00,NaN,1,16911
33574,Q20,2019,980500763,970407250,M3050,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.00,NaN,1,16910


In [14]:
Q20_doublon = Q20_id[(Q20_id.Nb_doublons_fiej > 1)]
Q20_2 = Q20_id[~(Q20_id.Nb_doublons_fiej > 1)]

In [ ]:
# On fait la moyenne pour toutes les lignes avec des doublons
### ATTENTION : exécuter cette cellule peut prendre quelques minutes
for i in Q20_doublon['id_fiej_perso'].unique():
    moyenne_EFFSALPLH = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['EFFSALPLH'].mean()
    moyenne_EFFSALPLF = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['EFFSALPLF'].mean()
    moyenne_EFFSALPAH = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['EFFSALPAH'].mean()
    moyenne_EFFSALPAF = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['EFFSALPAF'].mean()
    moyenne_ETPSALH = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETPSALH'].mean()
    moyenne_ETPSALF = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETPSALF'].mean()
    moyenne_ETP_PU = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETP_PU'].mean()
    moyenne_ETP_PH = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETP_PH'].mean()
    moyenne_ETP_AS = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETP_AS'].mean()
    moyenne_ETP_HU = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETP_HU'].mean()
    moyenne_ETP_AT = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETP_AT'].mean()
    moyenne_ETP_AU = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETP_AU'].mean()
    moyenne_EFFSAL = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['EFFSAL'].mean()
    moyenne_ETPSAL = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['ETPSAL'].mean()
    moyenne_EFFLIB = Q20_doublon[Q20_doublon['id_fiej_perso']==i]['EFFLIB'].mean()
    
    Indexs_Q20_doublon = Q20_doublon[Q20_doublon['id_fiej_perso']==i].index
    Q20_doublon['EFFSALPLH'][Indexs_Q20_doublon] = float(moyenne_EFFSALPLH)
    Q20_doublon['EFFSALPLF'][Indexs_Q20_doublon] = float(moyenne_EFFSALPLF)
    Q20_doublon['EFFSALPAH'][Indexs_Q20_doublon] = float(moyenne_EFFSALPAH)
    Q20_doublon['EFFSALPAF'][Indexs_Q20_doublon] = float(moyenne_EFFSALPAF)
    Q20_doublon['ETPSALH'][Indexs_Q20_doublon] = float(moyenne_ETPSALH)
    Q20_doublon['ETPSALF'][Indexs_Q20_doublon] = float(moyenne_ETPSALF)
    Q20_doublon['ETP_PU'][Indexs_Q20_doublon] = float(moyenne_ETP_PU)
    Q20_doublon['ETP_PH'][Indexs_Q20_doublon] = float(moyenne_ETP_PH)
    Q20_doublon['ETP_AS'][Indexs_Q20_doublon] = float(moyenne_ETP_AS)
    Q20_doublon['ETP_HU'][Indexs_Q20_doublon] = float(moyenne_ETP_HU)
    Q20_doublon['ETP_AT'][Indexs_Q20_doublon] = float(moyenne_ETP_AT)
    Q20_doublon['ETP_AU'][Indexs_Q20_doublon] = float(moyenne_ETP_AU)
    Q20_doublon['EFFSAL'][Indexs_Q20_doublon] = float(moyenne_EFFSAL)
    Q20_doublon['ETPSAL'][Indexs_Q20_doublon] = float(moyenne_ETPSAL)
    Q20_doublon['EFFLIB'][Indexs_Q20_doublon] = float(moyenne_EFFLIB)
    
Q20_doublon.head(10)

In [16]:
Q20_fiej_tot = pd.concat([Q20_doublon, Q20_2])

In [17]:
#On garde la première ligne
df_Q20_fiej = pd.DataFrame(Q20_fiej_tot)
df_Q20_fiej.drop_duplicates(subset= "id_fiej_perso", keep = 'first', inplace=True)

In [18]:
Q20_fiej =pd.merge(ISA, df_Q20_fiej, left_on = ['Numero_finess'], right_on = ['FI_EJ'],how='inner')
Q20_fiej

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFSALPLH,EFFSALPLF,...,ETP_PH,ETP_AS,ETP_HU,ETP_AT,ETP_AU,EFFSAL,ETPSAL,EFFLIB,Nb_doublons_fiej,id_fiej_perso
0,010008407,1.650548e+05,1,Q20,2019,010005239,010008407,M1010,3.666667,1.000000,...,1.086667,0.000000,0.0,0.126667,4.046667,6.000000,5.26,0.0,3,25871
1,010008407,1.650548e+05,1,Q20,2019,010005239,010008407,M3050,0.333333,1.333333,...,0.616667,0.333333,0.0,0.000000,0.000000,1.666667,0.95,0.0,3,25941
2,010008407,1.650548e+05,1,Q20,2019,010005239,010008407,M9999,9.000000,3.000000,...,7.286667,0.610000,0.0,0.750000,6.433333,20.000000,15.08,0.0,3,25942
3,010008407,1.650548e+05,1,Q20,2019,010005239,010008407,M1030,2.000000,0.000000,...,0.600000,0.830000,0.0,0.000000,3.900000,5.000000,5.33,0.0,1,16709
4,010008407,1.650548e+05,1,Q20,2019,010005239,010008407,M1040,0.000000,0.000000,...,0.600000,0.000000,0.0,0.000000,0.000000,1.000000,0.60,0.0,1,16804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10539,950110080,1.204192e+06,1,Q20,2019,950000364,950110080,M3030,1.000000,0.000000,...,1.000000,0.000000,0.0,0.700000,0.000000,6.000000,1.70,NaN,1,8224
10540,950110080,1.204192e+06,1,Q20,2019,950000364,950110080,M3040,1.000000,0.000000,...,1.000000,0.000000,0.0,0.000000,0.000000,1.000000,1.00,NaN,1,8222
10541,950110080,1.204192e+06,1,Q20,2019,950000364,950110080,M3050,5.000000,1.000000,...,6.000000,0.180000,0.0,0.000000,0.820000,7.000000,7.00,NaN,1,8221
10542,950110080,1.204192e+06,1,Q20,2019,950000364,950110080,M3070,2.000000,4.000000,...,3.000000,0.980000,0.0,0.230000,1.310000,9.000000,5.52,NaN,1,8220


In [19]:
Q20_good= pd.concat([Q20_fiej, Q20_fi] )
Q20_good

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFSALPLH,EFFSALPLF,...,ETP_PH,ETP_AS,ETP_HU,ETP_AT,ETP_AU,EFFSAL,ETPSAL,EFFLIB,Nb_doublons_fiej,id_fiej_perso
0,010008407,165054.798333,1,Q20,2019,010005239,010008407,M1010,3.666667,1.000000,...,1.086667,0.000000,0.0,0.126667,4.046667,6.000000,5.26,0.0,3.0,25871.0
1,010008407,165054.798333,1,Q20,2019,010005239,010008407,M3050,0.333333,1.333333,...,0.616667,0.333333,0.0,0.000000,0.000000,1.666667,0.95,0.0,3.0,25941.0
2,010008407,165054.798333,1,Q20,2019,010005239,010008407,M9999,9.000000,3.000000,...,7.286667,0.610000,0.0,0.750000,6.433333,20.000000,15.08,0.0,3.0,25942.0
3,010008407,165054.798333,1,Q20,2019,010005239,010008407,M1030,2.000000,0.000000,...,0.600000,0.830000,0.0,0.000000,3.900000,5.000000,5.33,0.0,1.0,16709.0
4,010008407,165054.798333,1,Q20,2019,010005239,010008407,M1040,0.000000,0.000000,...,0.600000,0.000000,0.0,0.000000,0.000000,1.000000,0.60,0.0,1.0,16804.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11016,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3011,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
11017,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
11018,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3040,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,1.00,NaN,NaN,NaN
11019,950807982,177318.826667,2,Q20,2019,950807982,950001636,M3050,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,2.000000,2.00,NaN,NaN,NaN


In [20]:
print("Il nous manque:", ISA[~ISA["Numero_finess"].isin(Q20_good["Numero_finess"])]["Numero_finess"].shape[0], "établissements.")

Il nous manque: 6 établissements.


Lorsque l'on effectue la jointure, 6 établissements présents sur la base ISA disparaissent et ne sont plus sur la nouvelle table. Nous devons les regarder, pour savoir à quoi est due cette disparition

- Observation des données manquantes : 

In [21]:
ISA_6 = ISA[~ISA["Numero_finess"].isin(Q20_good["Numero_finess"])]
ISA_6

,Numero_finess,Points_ISA,stjr
479,670020098,9344.040000,3
734,100000157,4551.390000,2
832,330780255,22251.530000,2
885,440000651,6613.990000,2
887,440024982,31024.516667,2
1076,780300364,14503.930000,2


En comparant avec la liste des établissements de santé dont les enquêtes sont facultatives, on retrouve de nombreux hopitaux. Je n'ai trouvé que la liste de 2014, disponible sur ce lien : https://www.fehap.fr/jcms/liste-des-etablissements-pour-lesquels-l-enquete-est-facultative-fehap_115923. Pour transférer cette base et pouvoir l'utiliser, il faut supprimer les deux premières lignes, et l'enregistrer au format CSV avec comme séparateur le point-virgule.

In [22]:
path_facult = r'D:/Données/ENSAE/2A/Stat_App/enquetes_facultatives_2014.csv'
facult=pd.read_csv(path_facult ,sep=';', encoding='latin-1')

In [23]:
facult

,Nbre,Finess PMSI,Nb séjours,Libellé Finess
0,1.0,010007987,8.0,CENTRE HOSPITALIER PUBLIC HAUTEVILLE-LOMPNES
1,2.0,010009132,21.0,CTRE HOSP INTERCOM AIN VAL DE SAONE
2,3.0,010780120,2.0,HOPITAL LOCAL DE MEXIMIEUX
3,4.0,010780138,12.0,HOPITAL LOCAL PONT-DE-VAUX
4,5.0,020000048,42.0,CTRE HOSP GERONTOLOGIQUE
...,...,...,...,...
312,313.0,970421038,13.0,CENTRE HOSPITALIER GABRIEL MARTIN
313,314.0,970462073,1.0,CLINIQUE DURIEUX
314,315.0,970462081,1.0,CLINIQUE DES ORCHIDÉES
315,316.0,970462107,5.0,CLINIQUE SAINTE CLOTILDE


In [24]:
ISA_facult=pd.merge(ISA_6, facult , left_on = ['Numero_finess'], right_on = ['Finess PMSI'],how='inner')
ISA_facult

,Numero_finess,Points_ISA,stjr,Nbre,Finess PMSI,Nb séjours,Libellé Finess
0,100000157,4551.39,2,37.0,100000157,57.0,CLINIQUE DES URSULINES


Sur les 6 établissements qui posent problèmes, 1 fait partie de ceux pour qui les enquêtes SAE étaient facultatives en 2014. On peut supposer qu'en 2019, ces enquêtes étaient encore facultatives pour cet établissements, et qu'il s'agit également de cela pour les 5 autres établissements.

In [25]:
# Voici les 5 établissements restants
ISA_5 = ISA_6[~ISA_6["Numero_finess"].isin(ISA_facult["Numero_finess"])]
ISA_5

,Numero_finess,Points_ISA,stjr
479,670020098,9344.040000,3
832,330780255,22251.530000,2
885,440000651,6613.990000,2
887,440024982,31024.516667,2
1076,780300364,14503.930000,2


**Analyse détaillée des établissements** : 
-  *670020098* :Groupement de coopération sanitaire de moyens - Exploitant, ouvert le 18 novembre 2019
- *330780255* : Etablissement de Soins Chirurgicaux : N° FINESS de l'entité juridique de rattachement :	33 001 005 9
- *440000651* : Etablissement de Soins Chirurgicaux : N° FINESS de l'entité juridique de rattachement :	44 000 121 2
- *440024982* : Etablissement de Soins Pluridisciplinaire : N° FINESS de l'entité juridique de rattachement :	33 000 004 3
- *780300364* : Etablissement de Soins Pluridisciplinaire : N° FINESS de l'entité juridique de rattachement :	78 000 367 97

Après vérification manuellement, aucun de ces établissements n'est dans le fichier Q20 de la base SAE.

Pour la suite, on fait l'hypothèse que ces établissements sont d'autres établissements qui n'ont pas à répondre à la partie Q20 de l'enquête SAE. Pour la base Q20, on peut donc garder ***Q20_good*** qui correspond à la base Q20 avec les différents numéro finess.

#### B) ISA et la base Q22

On utilise directement la méthode présenté au dessus pour avoir le même numéro finess avec la base Q22, présentant les données sur les internes.

In [26]:
Q22['FI'].nunique()

1072

 => on a 1072 numéro finess différents, soient autant que dans ISA MAIS si ca se trouve se sont des établissements qui ne font pas partis de notre champ d'étude

In [27]:
ISA['Numero_finess'].nunique()

1164

In [28]:
print("fi: ",Q22["FI"].size,'\n',"fi uniques :",Q22["FI"].nunique())

fi:  1072 
 fi uniques : 1072


In [29]:
# On retire les lignes identiques pour la fusion
df_Q22_fi = pd.DataFrame(Q22)
df_Q22_fi.drop_duplicates(subset ="FI", keep = 'first', inplace=True)

In [30]:
# On fusionne ISA et Q22, pour avoir les mêmes numéros FINESS
Q22_fi=pd.merge(ISA, df_Q22_fi, left_on = ['Numero_finess'], right_on = ['FI'],how='inner')
Q22_fi.head(5)

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,INTMED,INTPHA,INTODO,INTGEN,INTURG,FFI,INTTOT
0,060000528,356892.030000,3,Q22,2019,060000528,060780962,28.0,2.0,NaN,NaN,NaN,NaN,30
1,060780947,174552.540000,3,Q22,2019,060780947,060800174,41.0,NaN,NaN,6.0,NaN,NaN,47
2,060791811,146432.633333,3,Q22,2019,060791811,060010808,2.0,0.0,0.0,3.0,1.0,0.0,6
3,060794013,238173.260000,3,Q22,2019,060794013,060790797,4.0,NaN,NaN,NaN,NaN,NaN,4
4,130001647,854787.985000,3,Q22,2019,130001647,130784127,68.0,7.0,NaN,NaN,NaN,6.0,81


test en ne gardant que ceux qui n'ont pas été gardé dans le premier merge

In [31]:
print("fi_ej : ",Q22["FI_EJ"].size,'\n',"fi_ej uniques :",Q22["FI_EJ"].nunique())
# On remarque qu'il y a des doublons

fi_ej :  1072 
 fi_ej uniques : 743


In [ ]:
# Avant de supprimer les doublons, on fait la moyenne de leur valeurs pour qu'ils soient le plus représentatifs des hopitaux qui les concernent
for i in Q22['FI_EJ'].unique():
    moyenne_INTTOT = Q22[Q22['FI_EJ']==i]['INTTOT'].mean()
    moyenne_INTMED = Q22[Q22['FI_EJ']==i]['INTMED'].mean()
    moyenne_INTPHA = Q22[Q22['FI_EJ']==i]['INTPHA'].mean()
    moyenne_INTODO = Q22[Q22['FI_EJ']==i]['INTODO'].mean()
    moyenne_INTGEN = Q22[Q22['FI_EJ']==i]['INTGEN'].mean()
    moyenne_INTURG = Q22[Q22['FI_EJ']==i]['INTURG'].mean()
    moyenne_FFI = Q22[Q22['FI_EJ']==i]['FFI'].mean()
    Indexs_Q22 = Q22[Q22['FI_EJ']==i].index
    Q22['INTTOT'][Indexs_Q22] = float(moyenne_INTTOT)
    Q22['INTMED'][Indexs_Q22] = float(moyenne_INTMED)
    Q22['INTPHA'][Indexs_Q22] = float(moyenne_INTPHA)
    Q22['INTODO'][Indexs_Q22] = float(moyenne_INTODO)
    Q22['INTGEN'][Indexs_Q22] = float(moyenne_INTGEN)
    Q22['INTURG'][Indexs_Q22] = float(moyenne_INTURG)
    Q22['FFI'][Indexs_Q22] = float(moyenne_FFI)
Q22.head(10)

In [33]:
Q22.sort_values(by="FI_EJ").head(10)

,BOR,AN,FI,FI_EJ,INTMED,INTPHA,INTODO,INTGEN,INTURG,FFI,INTTOT
3,Q22,2019,010000198,010007987,NaN,NaN,NaN,1.0,NaN,NaN,1.0
0,Q22,2019,010000024,010780054,24.0,3.0,NaN,19.0,5.0,15.0,66.0
1,Q22,2019,010000032,010780062,NaN,NaN,NaN,5.0,NaN,NaN,5.0
2,Q22,2019,010000065,010780096,NaN,NaN,NaN,0.0,NaN,2.0,2.0
4,Q22,2019,010000495,010783009,4.0,NaN,NaN,NaN,NaN,3.0,7.0
5,Q22,2019,020000089,020000022,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6,Q22,2019,020000162,020000063,19.5,5.0,0.0,22.0,1.0,22.0,44.5
13,Q22,2019,020006664,020000063,19.5,5.0,0.0,22.0,1.0,22.0,44.5
7,Q22,2019,020000394,020000253,0.5,0.0,0.0,1.5,0.0,13.0,15.0
12,Q22,2019,020003877,020000253,0.5,0.0,0.0,1.5,0.0,13.0,15.0


In [34]:
# On retire les lignes identiques dans FIEJ pour la fusion
df_Q22_fiej = pd.DataFrame(Q22)
df_Q22_fiej.drop_duplicates(subset ="FI_EJ", keep = 'first', inplace=True)

In [35]:
Q22_fiej=pd.merge(ISA, df_Q22_fiej, left_on = ['Numero_finess'], right_on = ['FI_EJ'],how='inner')
Q22_fiej

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,INTMED,INTPHA,INTODO,INTGEN,INTURG,FFI,INTTOT
0,010780054,7.879291e+05,1,Q22,2019,010000024,010780054,24.0,3.0,NaN,19.0,5.0,15.0,66.0
1,010780062,1.881749e+05,1,Q22,2019,010000032,010780062,NaN,NaN,NaN,5.0,NaN,NaN,5.0
2,010780096,9.075654e+04,1,Q22,2019,010000065,010780096,NaN,NaN,NaN,0.0,NaN,2.0,2.0
3,020000022,2.967046e+04,1,Q22,2019,020000089,020000022,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,020000063,8.671006e+05,1,Q22,2019,020000162,020000063,19.5,5.0,0.0,22.0,1.0,22.0,44.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,950013870,7.774161e+05,1,Q22,2019,950000323,950013870,18.0,NaN,NaN,25.0,NaN,59.0,102.0
381,950015289,1.831389e+04,1,Q22,2019,950000349,950015289,NaN,NaN,NaN,3.0,NaN,NaN,3.0
382,950110015,1.079007e+06,1,Q22,2019,950000307,950110015,40.0,4.0,NaN,32.0,4.0,28.0,108.0
383,950110049,6.824362e+05,1,Q22,2019,950000331,950110049,13.0,0.0,0.0,28.0,5.0,50.0,96.0


In [36]:
# On a bien dans Q22_fiej je même nombre de ligne uniques 
Q22_fiej['Numero_finess'].nunique()

385

In [37]:
Q22_good= pd.concat([Q22_fiej, Q22_fi] )
Q22_good

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,INTMED,INTPHA,INTODO,INTGEN,INTURG,FFI,INTTOT
0,010780054,787929.068333,1,Q22,2019,010000024,010780054,24.0,3.0,NaN,19.0,5.0,15.0,66.0
1,010780062,188174.950000,1,Q22,2019,010000032,010780062,NaN,NaN,NaN,5.0,NaN,NaN,5.0
2,010780096,90756.543333,1,Q22,2019,010000065,010780096,NaN,NaN,NaN,0.0,NaN,2.0,2.0
3,020000022,29670.455000,1,Q22,2019,020000089,020000022,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,020000063,867100.583333,1,Q22,2019,020000162,020000063,19.5,5.0,0.0,22.0,1.0,22.0,44.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,920300365,56956.460000,2,Q22,2019,920300365,920815388,2.0,0.0,0.0,0.0,0.0,NaN,2.0
160,920300753,425375.070000,2,Q22,2019,920300753,920810736,4.0,NaN,NaN,NaN,NaN,NaN,4.0
161,940006679,102715.460000,2,Q22,2019,940006679,940017338,1.0,NaN,NaN,NaN,NaN,NaN,1.0
162,940300270,119607.230000,2,Q22,2019,940300270,940000771,1.0,NaN,NaN,NaN,NaN,NaN,1.0


In [38]:
Q22_good["Numero_finess"].nunique()

549

In [39]:
print("Il nous manque:", ISA[~ISA["Numero_finess"].isin(Q22_good["Numero_finess"])]["Numero_finess"].shape[0], "établissements.")

Il nous manque: 615 établissements.


 On n'a que 549 établissements lorsque l'on joint les bases ISA et Q22, soit moins de la moitié par rapport à ceux de la base ISA.
**QUESTION MILCENT INTERNES DANS LES HOPITAUX**

In [40]:
ISA[~ISA["Numero_finess"].isin(Q22_good["Numero_finess"])]["Numero_finess"]

0       010008407
1       010009132
6       020000048
7       020000055
9       020000071
          ...    
1173    950300244
1174    950300277
1175    950300301
1176    950300350
1177    950807982
Name: Numero_finess, Length: 615, dtype: object

 Cela est sûrement dû au fait que tous les établissements n'accueillent pas des internes (étudiants). En 
 effet, en se penchant sur la documentation : https://drees.solidarites-sante.gouv.fr/sites/default/files/2021-03/6-8.pdf, 
 On ne trouve pas trop de réponse à nos questions.
 

### C) ISA et la base Q23

#### - Plusieurs  problème dans les données, avant même de les joindre avec la base ISA
- le type : certaines lignes de FI et FI_EJ sont en str, d'autres en float : il faut donc les remplacer
- certains établissements de FI_EJ sont codés  avec un .0, comme ci-dessous

- le type

In [41]:
ISA[(ISA.Numero_finess== "920110020")]

,Numero_finess,Points_ISA,stjr
671,920110020,190013.328333,1


In [42]:
Q23[(Q23.FI== "920120011")].head(5)

,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM


In [43]:
Q23[['FI','FI_EJ']] = Q23[['FI','FI_EJ']].astype(str)

In [44]:
Q23[(Q23.FI== "920120011")].head(5)

,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM
70546,Q23,2019,920120011,750821092.0,N1000,NaN,NaN,26.0,26.0,220.91,228.0,254.0,246.91
70547,Q23,2019,920120011,750821092.0,N1100,NaN,NaN,NaN,NaN,13.05,14.0,14.0,13.05
70548,Q23,2019,920120011,750821092.0,N1210,NaN,NaN,1.0,1.0,91.60,95.0,96.0,92.60
70549,Q23,2019,920120011,750821092.0,N1220,NaN,NaN,25.0,25.0,115.26,118.0,143.0,140.26
70550,Q23,2019,920120011,750821092.0,N1400,NaN,NaN,NaN,NaN,1.00,1.0,1.0,1.00


- le cas des  ".0"

In [45]:
Q23[(Q23.FI_EJ== "920110020")].head(5)

,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM


In [46]:
Q23[(Q23.FI_EJ== "920110020.0")].head(5)

,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM
69707,Q23,2019,920000577,920110020.0,N1000,36.0,37.10,26.0,17.51,111.95,111.0,173.0,166.56
69708,Q23,2019,920000577,920110020.0,N1100,0.0,0.53,1.0,0.15,6.53,6.0,7.0,7.21
69709,Q23,2019,920000577,920110020.0,N1210,4.0,4.89,6.0,3.45,20.16,20.0,30.0,28.50
69710,Q23,2019,920000577,920110020.0,N1220,32.0,31.68,19.0,13.91,84.26,84.0,135.0,129.85
69711,Q23,2019,920000577,920110020.0,N1400,0.0,0.00,0.0,0.00,1.00,1.0,1.0,1.00


In [47]:
# On ne garde que les 9 premiers caractères : 
Q23['FI_EJ'] = Q23["FI_EJ"].str[:9]

#### -  merge entre Numero_finess et FI

In [48]:
Q23_short_fi = Q23[["FI","PERSO","AN"]]
Q23_short_fi.groupby(by=["FI", "PERSO"]).count().sort_values(by="AN")
Q23_group_fi = Q23_short_fi.groupby(by=["FI", "PERSO"]).count().sort_values(by="AN")
Q23_group_fi = Q23_group_fi.rename(columns = {'AN': 'Nb_doublons_fi'})

Q23_group_fi = Q23_group_fi.reset_index()

#On réindexe, pour avoir une colonne avec les identifiants de chaque intervalle, 
#qu'on appelle id_int pour identifiant intervalle
Q23_group_fi['id_fi_perso'] = Q23_group_fi.index
Q23_group_fi.sort_values(by="Nb_doublons_fi")
Q23_group_fi

,FI,PERSO,Nb_doublons_fi,id_fi_perso
0,010000024,N1000,1,0
1,690010848,N2430,1,1
2,690010848,N2410,1,2
3,690010848,N2210,1,3
4,690010848,N2130,1,4
...,...,...,...,...
77315,360000137,N1220,1,77315
77316,360000137,N1210,1,77316
77317,360000137,N1100,1,77317
77318,360000129,N9999,1,77318


In [49]:
print("fi : ",Q23_group_fi["id_fi_perso"].size,'\n',"fi uniques :",Q23_group_fi["id_fi_perso"].nunique())
# On remarque qu'il n'y a pas de doublons pour FI

fi :  77320 
 fi uniques : 77320


In [50]:
Q23_fi=pd.merge(ISA, Q23, left_on = ['Numero_finess'], right_on = ['FI'], how='inner')
Q23_fi

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM
0,060000528,356892.030000,3,Q23,2019,060000528,060780962,N1000,142.0,133.31,7.0,10.04,2.61,2.0,151.0,145.96
1,060000528,356892.030000,3,Q23,2019,060000528,060780962,N1100,4.0,4.09,NaN,NaN,1.00,1.0,5.0,5.09
2,060000528,356892.030000,3,Q23,2019,060000528,060780962,N1210,63.0,56.88,6.0,5.86,NaN,NaN,69.0,62.74
3,060000528,356892.030000,3,Q23,2019,060000528,060780962,N1220,74.0,71.70,1.0,4.18,1.61,1.0,76.0,77.49
4,060000528,356892.030000,3,Q23,2019,060000528,060780962,N1400,1.0,0.64,NaN,NaN,NaN,NaN,1.0,0.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14008,950807982,177318.826667,2,Q23,2019,950807982,950001636,N4160,2.0,2.00,NaN,NaN,NaN,NaN,2.0,2.00
14009,950807982,177318.826667,2,Q23,2019,950807982,950001636,N5000,6.0,6.00,NaN,NaN,NaN,NaN,6.0,6.00
14010,950807982,177318.826667,2,Q23,2019,950807982,950001636,N511A,4.0,4.00,NaN,NaN,NaN,NaN,4.0,4.00
14011,950807982,177318.826667,2,Q23,2019,950807982,950001636,N5120,2.0,2.00,NaN,NaN,NaN,NaN,2.0,2.00


In [51]:
Q23_fi[(Q23_fi.FI== "920120011")]

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM
4484,920120011,709365.153333,1,Q23,2019,920120011,750821092,N1000,NaN,NaN,26.0,26.0,220.91,228.0,254.0,246.91
4485,920120011,709365.153333,1,Q23,2019,920120011,750821092,N1100,NaN,NaN,NaN,NaN,13.05,14.0,14.0,13.05
4486,920120011,709365.153333,1,Q23,2019,920120011,750821092,N1210,NaN,NaN,1.0,1.0,91.60,95.0,96.0,92.60
4487,920120011,709365.153333,1,Q23,2019,920120011,750821092,N1220,NaN,NaN,25.0,25.0,115.26,118.0,143.0,140.26
4488,920120011,709365.153333,1,Q23,2019,920120011,750821092,N1400,NaN,NaN,NaN,NaN,1.00,1.0,1.0,1.00
4489,920120011,709365.153333,1,Q23,2019,920120011,750821092,N2000,NaN,NaN,32.0,32.0,835.70,858.0,890.0,867.70
4490,920120011,709365.153333,1,Q23,2019,920120011,750821092,N2120,NaN,NaN,NaN,NaN,19.30,22.0,22.0,19.30
4491,920120011,709365.153333,1,Q23,2019,920120011,750821092,N2130,NaN,NaN,NaN,NaN,21.85,24.0,24.0,21.85
4492,920120011,709365.153333,1,Q23,2019,920120011,750821092,N2210,NaN,NaN,4.0,4.0,328.50,333.0,337.0,332.50
4493,920120011,709365.153333,1,Q23,2019,920120011,750821092,N2220,NaN,NaN,NaN,NaN,7.00,7.0,7.0,7.00


In [52]:
Q23_short_fiej= Q23[["FI_EJ","PERSO","AN"]]

In [53]:
Q23_short_fiej.groupby(by=["FI_EJ", "PERSO"]).count().sort_values(by="AN") 
# On remarque tout de suite que certaines lignes sont des doublons.

AN
FI_EJ     PERSO    
010000156 N1000   1
660000373 N1210   1
          N1220   1
          N2000   1
          N2120   1
...              ..
920030269 N2510  75
          N1100  76
          N2000  76
          N9999  76
          N1000  76

[51128 rows x 1 columns]

In [54]:
Q23_group_fiej = Q23_short_fiej.groupby(by=["FI_EJ", "PERSO"]).count().sort_values(by="AN")
Q23_group_fiej = Q23_group_fiej.rename(columns = {'AN': 'Nb_doublons_fiej'})

Q23_group_fiej = Q23_group_fiej.reset_index()

#On réindexe, pour avoir une colonne avec les identifiants de chaque intervalle, 
#qu'on appelle id_fiej_perso pour identifier intervalle
Q23_group_fiej['id_fiej_perso'] = Q23_group_fiej.index
Q23_group_fiej.sort_values(by="Nb_doublons_fiej")
Q23_group_fiej


,FI_EJ,PERSO,Nb_doublons_fiej,id_fiej_perso
0,010000156,N1000,1,0
1,660000373,N1210,1,1
2,660000373,N1220,1,2
3,660000373,N2000,1,3
4,660000373,N2120,1,4
...,...,...,...,...
51123,920030269,N2510,75,51123
51124,920030269,N1100,76,51124
51125,920030269,N2000,76,51125
51126,920030269,N9999,76,51126


In [55]:
print("On a", Q23_group_fiej[(Q23_group_fiej.Nb_doublons_fiej > 1)].shape[0], "cas où il y plusieurs lignes pour un même type de métier d'un même hôpital")

On a 10233 cas où il y plusieurs lignes pour un même type de métier d'un même hôpital


In [56]:
Q23_id = pd.merge(Q23, Q23_group_fiej, left_on = ["FI_EJ", "PERSO"], right_on = ["FI_EJ", "PERSO"], how="inner")
Q23_id

,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM,Nb_doublons_fiej,id_fiej_perso
0,Q23,2019,010000024,010780054,N1000,17.0,17.34,47.0,31.22,185.41,199.0,263.0,233.97,2,44698
1,Q23,2019,010009314,010780054,N1000,NaN,NaN,NaN,NaN,3.24,5.0,5.0,3.24,2,44698
2,Q23,2019,010000024,010780054,N1100,1.0,1.10,NaN,NaN,5.19,6.0,7.0,6.29,1,29865
3,Q23,2019,010000024,010780054,N1210,NaN,0.17,3.0,3.86,95.56,99.0,102.0,99.59,2,44697
4,Q23,2019,010009314,010780054,N1210,NaN,NaN,NaN,NaN,2.84,3.0,3.0,2.84,2,44697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77315,Q23,2019,980501258,970407250,N5130,NaN,0.57,NaN,NaN,NaN,NaN,NaN,0.57,4,49049
77316,Q23,2019,980500763,970407250,N9999,22.0,13.50,18.0,12.81,NaN,NaN,40.0,26.31,4,49048
77317,Q23,2019,980500920,970407250,N9999,NaN,7.23,NaN,5.23,NaN,NaN,NaN,12.46,4,49048
77318,Q23,2019,980501159,970407250,N9999,NaN,3.30,NaN,0.43,NaN,NaN,NaN,3.73,4,49048


In [57]:
Q23_doublon = Q23_id[(Q23_id.Nb_doublons_fiej > 1)]
Q23_2 = Q23_id[~(Q23_id.Nb_doublons_fiej > 1)]

In [ ]:
#moyenne que pour les doublons, sinon ça prend trop de temps
for i in Q23_doublon['id_fiej_perso'].unique():
    moyenne_EFFCDI = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['EFFCDI'].mean()
    moyenne_ETPCDI = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['ETPCDI'].mean()
    moyenne_EFFCDD = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['EFFCDD'].mean()
    moyenne_ETPCDD = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['ETPCDD'].mean()
    moyenne_ETPTIT = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['ETPTIT'].mean()
    moyenne_EFFTIT = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['EFFTIT'].mean()
    moyenne_EFFPNM = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['EFFPNM'].mean()
    moyenne_ETPPNM = Q23_doublon[Q23_doublon['id_fiej_perso']==i]['ETPPNM'].mean()
    
    Indexs_Q23_doublon = Q23_doublon[Q23_doublon['id_fiej_perso']==i].index
    Q23_doublon['EFFCDI'][Indexs_Q23_doublon] = float(moyenne_EFFCDI)
    Q23_doublon['ETPCDI'][Indexs_Q23_doublon] = float(moyenne_ETPCDI)
    Q23_doublon['EFFCDD'][Indexs_Q23_doublon] = float(moyenne_EFFCDD)
    Q23_doublon['ETPCDD'][Indexs_Q23_doublon] = float(moyenne_ETPCDD)
    Q23_doublon['ETPTIT'][Indexs_Q23_doublon] = float(moyenne_ETPTIT)
    Q23_doublon['EFFTIT'][Indexs_Q23_doublon] = float(moyenne_EFFTIT)
    Q23_doublon['EFFPNM'][Indexs_Q23_doublon] = float(moyenne_EFFPNM)
    Q23_doublon['ETPPNM'][Indexs_Q23_doublon] = float(moyenne_ETPPNM)
    
Q23_doublon.head(10)

In [59]:
Q23_fiej_tot = pd.concat([Q23_doublon, Q23_2])

In [60]:
#On garde la première ligne
df_Q23_fiej = pd.DataFrame(Q23_fiej_tot)
df_Q23_fiej.drop_duplicates(subset= "id_fiej_perso", keep = 'first', inplace=True)

In [61]:
Q23_fiej =pd.merge(ISA, df_Q23_fiej, left_on = ['Numero_finess'], right_on = ['FI_EJ'],how='inner')
Q23_fiej

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM,Nb_doublons_fiej,id_fiej_perso
0,010008407,1.650548e+05,1,Q23,2019,010005239,010008407,N1000,1.000000,1.153333,4.666667,3.650000,16.393333,17.666667,23.333333,21.196667,3,47048
1,010008407,1.650548e+05,1,Q23,2019,010005239,010008407,N1210,0.000000,0.000000,0.000000,0.000000,7.955000,8.000000,8.000000,7.955000,2,44861
2,010008407,1.650548e+05,1,Q23,2019,010005239,010008407,N1220,1.000000,1.153333,7.000000,3.650000,10.156667,11.666667,17.333333,14.960000,3,47061
3,010008407,1.650548e+05,1,Q23,2019,010005239,010008407,N2000,1.333333,1.826667,14.666667,12.946667,82.550000,94.666667,110.666667,97.323333,3,47053
4,010008407,1.650548e+05,1,Q23,2019,010005239,010008407,N2120,0.000000,0.000000,0.000000,0.043333,2.443333,3.000000,3.000000,2.486667,3,47113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,950110080,1.204192e+06,1,Q23,2019,950000364,950110080,N5110,21.000000,18.730000,NaN,0.500000,2.980000,3.000000,24.000000,22.210000,1,16265
16252,950110080,1.204192e+06,1,Q23,2019,950000364,950110080,N511A,19.000000,18.900000,11.000000,10.330000,7.970000,10.000000,40.000000,37.200000,1,16257
16253,950110080,1.204192e+06,1,Q23,2019,950000364,950110080,N5130,19.000000,13.930000,22.000000,25.460000,126.530000,123.000000,164.000000,165.920000,1,16258
16254,950110080,1.204192e+06,1,Q23,2019,950000364,950110080,N5140,4.000000,4.000000,3.000000,2.740000,13.090000,13.000000,20.000000,19.830000,1,16259


In [62]:
Q23_good= pd.concat([Q23_fiej, Q23_fi] )
Q23_good

,Numero_finess,Points_ISA,stjr,BOR,AN,FI,FI_EJ,PERSO,EFFCDI,ETPCDI,EFFCDD,ETPCDD,ETPTIT,EFFTIT,EFFPNM,ETPPNM,Nb_doublons_fiej,id_fiej_perso
0,010008407,165054.798333,1,Q23,2019,010005239,010008407,N1000,1.000000,1.153333,4.666667,3.650000,16.393333,17.666667,23.333333,21.196667,3.0,47048.0
1,010008407,165054.798333,1,Q23,2019,010005239,010008407,N1210,0.000000,0.000000,0.000000,0.000000,7.955000,8.000000,8.000000,7.955000,2.0,44861.0
2,010008407,165054.798333,1,Q23,2019,010005239,010008407,N1220,1.000000,1.153333,7.000000,3.650000,10.156667,11.666667,17.333333,14.960000,3.0,47061.0
3,010008407,165054.798333,1,Q23,2019,010005239,010008407,N2000,1.333333,1.826667,14.666667,12.946667,82.550000,94.666667,110.666667,97.323333,3.0,47053.0
4,010008407,165054.798333,1,Q23,2019,010005239,010008407,N2120,0.000000,0.000000,0.000000,0.043333,2.443333,3.000000,3.000000,2.486667,3.0,47113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14008,950807982,177318.826667,2,Q23,2019,950807982,950001636,N4160,2.000000,2.000000,NaN,NaN,NaN,NaN,2.000000,2.000000,NaN,NaN
14009,950807982,177318.826667,2,Q23,2019,950807982,950001636,N5000,6.000000,6.000000,NaN,NaN,NaN,NaN,6.000000,6.000000,NaN,NaN
14010,950807982,177318.826667,2,Q23,2019,950807982,950001636,N511A,4.000000,4.000000,NaN,NaN,NaN,NaN,4.000000,4.000000,NaN,NaN
14011,950807982,177318.826667,2,Q23,2019,950807982,950001636,N5120,2.000000,2.000000,NaN,NaN,NaN,NaN,2.000000,2.000000,NaN,NaN


In [63]:
print("Il nous manque", ISA[~ISA["Numero_finess"].isin(Q23_good["Numero_finess"])]["Numero_finess"].shape[0], "établissements.")

Il nous manque 7 établissements.


In [64]:
ISA[~ISA["Numero_finess"].isin(Q23_good["Numero_finess"])].head(55)

,Numero_finess,Points_ISA,stjr
339,520004680,126991.370000,3
340,520004714,77375.870000,3
375,560029068,129285.330000,3
479,670020098,9344.040000,3
632,840019053,2536.030000,3
885,440000651,6613.990000,2
887,440024982,31024.516667,2


Après vérification, aucun de ces établissements n'est dans la base Q23 de la SAE

### 2) Regroupement des bases en un unique dataframe

On souhaite vérifier que les hopitaux contenus Q22 et Q23 sont également dans Q20, et sinon regarder plus précisément de quel établissement il s'agit.

#### A) Traitement des trois Q20_good, Q22_good, Q23_good avant le merge

In [65]:
Q20_good.columns

Index(['Numero_finess', 'Points_ISA', 'stjr', 'BOR', 'AN', 'FI', 'FI_EJ',
       'PERSO', 'EFFSALPLH', 'EFFSALPLF', 'EFFSALPAH', 'EFFSALPAF',
       'EFFLIBPLH', 'EFFLIBPLF', 'EFFLIBPAH', 'EFFLIBPAF', 'ETPSALH',
       'ETPSALF', 'ETP_PU', 'ETP_PH', 'ETP_AS', 'ETP_HU', 'ETP_AT', 'ETP_AU',
       'EFFSAL', 'ETPSAL', 'EFFLIB', 'Nb_doublons_fiej', 'id_fiej_perso'],
      dtype='object')

In [66]:
Q22_good.columns

Index(['Numero_finess', 'Points_ISA', 'stjr', 'BOR', 'AN', 'FI', 'FI_EJ',
       'INTMED', 'INTPHA', 'INTODO', 'INTGEN', 'INTURG', 'FFI', 'INTTOT'],
      dtype='object')

In [67]:
Q22_good.columns

Index(['Numero_finess', 'Points_ISA', 'stjr', 'BOR', 'AN', 'FI', 'FI_EJ',
       'INTMED', 'INTPHA', 'INTODO', 'INTGEN', 'INTURG', 'FFI', 'INTTOT'],
      dtype='object')

In [68]:
#On ne garde que quelques colonnes, pour mieux joindre les bases de données après
Q20_good = Q20_good[["Numero_finess","PERSO","EFFSAL","EFFLIB"]]
Q20_good = Q20_good[Q20_good['PERSO'] != "M9999"] #On supprime la catégorie "total", qui regroupe l'ensemble du personnel médical
Q22_good = Q22_good[["Numero_finess","INTTOT"]]
Q23_good = Q23_good[["Numero_finess","PERSO","EFFPNM"]]
ISA = ISA[["Numero_finess","Points_ISA","stjr"]]

- **Q20_good**

On souhaite que la colonne Perso soit verticale, pour pouvoir avoir toutes les informations d'un établissement sur la même ligne

In [69]:
Q20_good_pivot = Q20_good.pivot_table(index='Numero_finess', columns='PERSO', values=['EFFLIB', 'EFFSAL'])
Q20_good_pivot

EFFLIB                                                        \
PERSO          M1010 M1020 M1030 M1031 M1040 M1050 M1070 M1090 M1100 M1110   
Numero_finess                                                                
010007300        NaN   NaN   6.0   NaN   1.0   NaN   NaN   1.0   NaN   NaN   
010008407        0.0   NaN   0.0   NaN   0.0   NaN   NaN   0.0   NaN   NaN   
010009132        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
010780054        7.0   NaN   NaN   NaN   2.0   NaN   NaN   NaN   NaN   NaN   
010780062        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...              ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
950300277        NaN   2.0   6.0   NaN  10.0   1.0   2.0   6.0   NaN   6.0   
950300301        NaN   NaN   NaN   NaN   2.0   NaN   1.0   NaN   NaN   NaN   
950300350        NaN   NaN   1.0   NaN   3.0   NaN   NaN   1.0   NaN   3.0   
950630012        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
950807982        NaN   NaN  14.0   NaN   7.0   NaN   1.0  13.0   1.0   3.0   

               ... EFFSAL                                                 \
PERSO          ...  M2140 M2150 M3011 M3012 M3020 M3030 M3040      M3050   
Numero_finess  ...                                                         
010007300      ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN        NaN   
010008407      ...    NaN   NaN   2.0   NaN   NaN   NaN   NaN   1.666667   
010009132      ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.000000   
010780054      ...    NaN   NaN   NaN   6.0   NaN   2.0   2.0  11.000000   
010780062      ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   3.000000   
...            ...    ...   ...   ...   ...   ...   ...   ...        ...   
950300277      ...    NaN   NaN   NaN   NaN   NaN   NaN   1.0   2.000000   
950300301      ...    NaN   NaN   NaN   NaN   1.0   NaN   1.0   1.000000   
950300350      ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.000000   
950630012      ...    NaN   NaN   NaN   NaN   NaN   NaN   1.0   2.000000   
950807982      ...    NaN   NaN   NaN   NaN   NaN   NaN   1.0   2.000000   

                           
PERSO         M3060 M3070  
Numero_finess              
010007300       NaN   NaN  
010008407       NaN   NaN  
010009132       NaN   NaN  
010780054       NaN   NaN  
010780062       NaN   NaN  
...             ...   ...  
950300277       NaN   NaN  
950300301       NaN   NaN  
950300350       NaN   NaN  
950630012       NaN   NaN  
950807982       NaN   NaN  

[1154 rows x 92 columns]

- **Q23_good**

In [70]:
Q23_good_pivot = Q23_good.pivot_table(index='Numero_finess', columns='PERSO', values=['EFFPNM'])
Q23_good_pivot

EFFPNM                                                 \
PERSO               N1000 N1100 N1210      N1220 N1300 N1400       N2000   
Numero_finess                                                              
010007300        3.000000   1.0   2.0        NaN   NaN   NaN    8.000000   
010008407       23.333333   2.0   8.0  17.333333   NaN   0.0  110.666667   
010009132        9.500000   1.5   3.5   4.500000   NaN   NaN   73.500000   
010780054      134.000000   7.0  52.5  77.500000   NaN   1.0  426.333333   
010780062       62.000000   2.0  10.0  50.000000   NaN   NaN  312.000000   
...                   ...   ...   ...        ...   ...   ...         ...   
950300277       96.000000   9.0  38.0  48.000000   NaN   1.0  397.000000   
950300301       28.000000   3.0  10.0  14.000000   NaN   1.0  150.000000   
950300350        8.000000   NaN   3.0   5.000000   NaN   NaN   29.000000   
950630012       28.000000   2.0   6.0  19.000000   NaN   1.0  188.000000   
950807982       72.000000   3.0   6.0  63.000000   NaN   NaN  370.000000   

                                            ...                           \
PERSO              N2120 N2130       N2210  ... N4170 N4180 N4190  N5000   
Numero_finess                               ...                            
010007300            NaN   NaN    6.000000  ...   NaN   NaN   NaN    NaN   
010008407       3.000000   NaN   38.333333  ...   NaN   NaN   NaN   28.0   
010009132       3.000000   NaN   16.000000  ...   NaN   NaN   NaN   17.5   
010780054      13.333333   NaN  166.333333  ...   NaN   2.0   NaN  116.0   
010780062      12.000000   2.0   98.000000  ...   NaN   NaN   NaN   61.0   
...                  ...   ...         ...  ...   ...   ...   ...    ...   
950300277       8.000000   NaN  158.000000  ...   NaN   NaN   NaN   16.0   
950300301       5.000000   NaN   48.000000  ...   NaN   NaN   NaN    3.0   
950300350       1.000000   NaN   18.000000  ...   NaN   NaN   NaN    1.0   
950630012      10.000000   1.0   62.000000  ...   NaN   NaN   NaN   14.0   
950807982       8.000000   4.0  183.000000  ...   NaN   NaN   NaN    6.0   

                                                         
PERSO         N5110 N511A N5120 N5130 N5140       N9999  
Numero_finess                                            
010007300       NaN   NaN   NaN   NaN   NaN   11.000000  
010008407       2.0   7.0   NaN  22.5   2.0  165.333333  
010009132       1.5   3.0   NaN  13.0   NaN  104.500000  
010780054      10.0  34.0   4.0  86.0  12.0  635.000000  
010780062       6.0   6.0   NaN  42.0   7.0  473.000000  
...             ...   ...   ...   ...   ...         ...  
950300277       NaN   3.0   1.0  12.0   NaN  520.000000  
950300301       NaN   3.0   NaN   NaN   NaN  186.000000  
950300350       NaN   1.0   NaN   NaN   NaN   41.000000  
950630012       NaN   2.0   1.0  10.0   1.0  278.000000  
950807982       NaN   4.0   2.0   NaN   NaN  459.000000  

[1152 rows x 53 columns]

#### 2) On merge nos données pour obtenir une unique base : 

- **la base complète**

In [71]:
# On merge d'abord ceux avec un pivot
result1 = pd.merge(Q20_good_pivot, Q23_good_pivot, on="Numero_finess", how ="outer")

# Puis on ajoute ISA pour avoir les points ISA
result2 = pd.merge(result1, ISA, on="Numero_finess" )

C:\Users\Vinciane\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [72]:
#On ajoute les internes
VAR_EXPL = pd.merge(result2, Q22_good, on=["Numero_finess"], how= "outer" )
VAR_EXPL

,Numero_finess,"(EFFLIB, M1010)","(EFFLIB, M1020)","(EFFLIB, M1030)","(EFFLIB, M1031)","(EFFLIB, M1040)","(EFFLIB, M1050)","(EFFLIB, M1070)","(EFFLIB, M1090)","(EFFLIB, M1100)",...,"(EFFPNM, N5000)","(EFFPNM, N5110)","(EFFPNM, N511A)","(EFFPNM, N5120)","(EFFPNM, N5130)","(EFFPNM, N5140)","(EFFPNM, N9999)",Points_ISA,stjr,INTTOT
0,010007300,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,3785.960000,2,NaN
1,010008407,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,...,28.0,2.0,7.0,NaN,22.5,2.0,165.333333,165054.798333,1,NaN
2,010009132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.5,1.5,3.0,NaN,13.0,NaN,104.500000,5715.170000,1,NaN
3,010780054,7.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,116.0,10.0,34.0,4.0,86.0,12.0,635.000000,787929.068333,1,66.0
4,010780062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.0,6.0,6.0,NaN,42.0,7.0,473.000000,188174.950000,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,950630012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,NaN,2.0,1.0,10.0,1.0,278.000000,20931.220000,3,1.0
1153,950807982,NaN,NaN,14.0,NaN,7.0,NaN,1.0,13.0,1.0,...,6.0,NaN,4.0,2.0,NaN,NaN,459.000000,177318.826667,2,NaN
1154,330780255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.0,0.0,NaN,0.000000,22251.530000,2,NaN
1155,450000237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,0.000000,43416.985000,2,NaN


In [73]:
VAR_EXPL.head(50)

,Numero_finess,"(EFFLIB, M1010)","(EFFLIB, M1020)","(EFFLIB, M1030)","(EFFLIB, M1031)","(EFFLIB, M1040)","(EFFLIB, M1050)","(EFFLIB, M1070)","(EFFLIB, M1090)","(EFFLIB, M1100)",...,"(EFFPNM, N5000)","(EFFPNM, N5110)","(EFFPNM, N511A)","(EFFPNM, N5120)","(EFFPNM, N5130)","(EFFPNM, N5140)","(EFFPNM, N9999)",Points_ISA,stjr,INTTOT
0,010007300,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,3.785960e+03,2,NaN
1,010008407,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,...,28.0,2.00,7.0,NaN,22.5,2.000000,165.333333,1.650548e+05,1,NaN
2,010009132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.5,1.50,3.0,NaN,13.0,NaN,104.500000,5.715170e+03,1,NaN
3,010780054,7.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,116.0,10.00,34.0,4.0,86.0,12.000000,635.000000,7.879291e+05,1,66.0
4,010780062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.0,6.00,6.0,NaN,42.0,7.000000,473.000000,1.881749e+05,1,5.0
5,010780096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.5,1.00,4.5,NaN,10.5,NaN,155.000000,9.075654e+04,1,2.0
6,010780195,2.0,NaN,12.0,NaN,10.0,NaN,NaN,3.0,NaN,...,12.0,NaN,9.0,1.0,2.0,NaN,294.000000,1.792710e+05,2,NaN
7,010780203,8.0,NaN,4.0,NaN,NaN,NaN,1.0,2.0,NaN,...,6.0,NaN,1.0,NaN,NaN,5.000000,223.000000,1.224635e+05,2,NaN
8,020000022,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,21.0,NaN,3.0,NaN,18.0,NaN,209.000000,2.967046e+04,1,1.0
9,020000048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.0,1.00,NaN,2.0,15.0,NaN,143.000000,2.097850e+04,1,NaN


In [74]:
# On voit que les erreurs sont liées à INTTOT => on reregarde notre importation des données

In [75]:
VAR_EXPL.groupby(by="Numero_finess").sum()

,"(EFFLIB, M1010)","(EFFLIB, M1020)","(EFFLIB, M1030)","(EFFLIB, M1031)","(EFFLIB, M1040)","(EFFLIB, M1050)","(EFFLIB, M1070)","(EFFLIB, M1090)","(EFFLIB, M1100)","(EFFLIB, M1110)",...,"(EFFPNM, N5000)","(EFFPNM, N5110)","(EFFPNM, N511A)","(EFFPNM, N5120)","(EFFPNM, N5130)","(EFFPNM, N5140)","(EFFPNM, N9999)",Points_ISA,stjr,INTTOT
Numero_finess,,,,,,,,,,,,,,,,,,,,,
010007300,0.0,0.0,6.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.000000,3785.960000,2,0.0
010008407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,2.0,7.0,0.0,22.5,2.0,165.333333,165054.798333,1,0.0
010009132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.5,1.5,3.0,0.0,13.0,0.0,104.500000,5715.170000,1,0.0
010780054,7.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,116.0,10.0,34.0,4.0,86.0,12.0,635.000000,787929.068333,1,66.0
010780062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.0,6.0,6.0,0.0,42.0,7.0,473.000000,188174.950000,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950300277,0.0,2.0,6.0,0.0,10.0,1.0,2.0,6.0,0.0,6.0,...,16.0,0.0,3.0,1.0,12.0,0.0,520.000000,140779.191667,2,0.0
950300301,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,...,3.0,0.0,3.0,0.0,0.0,0.0,186.000000,6779.931667,2,0.0
950300350,0.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,3.0,...,1.0,0.0,1.0,0.0,0.0,0.0,41.000000,11099.390000,2,0.0


In [76]:
VAR_EXPL_GROUP = VAR_EXPL.groupby(by="Numero_finess").sum()

In [77]:
VAR_EXPL_GROUP.reset_index()

,Numero_finess,"(EFFLIB, M1010)","(EFFLIB, M1020)","(EFFLIB, M1030)","(EFFLIB, M1031)","(EFFLIB, M1040)","(EFFLIB, M1050)","(EFFLIB, M1070)","(EFFLIB, M1090)","(EFFLIB, M1100)",...,"(EFFPNM, N5000)","(EFFPNM, N5110)","(EFFPNM, N511A)","(EFFPNM, N5120)","(EFFPNM, N5130)","(EFFPNM, N5140)","(EFFPNM, N9999)",Points_ISA,stjr,INTTOT
0,010007300,0.0,0.0,6.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.000000,3785.960000,2,0.0
1,010008407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,2.0,7.0,0.0,22.5,2.0,165.333333,165054.798333,1,0.0
2,010009132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.5,1.5,3.0,0.0,13.0,0.0,104.500000,5715.170000,1,0.0
3,010780054,7.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,116.0,10.0,34.0,4.0,86.0,12.0,635.000000,787929.068333,1,66.0
4,010780062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.0,6.0,6.0,0.0,42.0,7.0,473.000000,188174.950000,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,950300277,0.0,2.0,6.0,0.0,10.0,1.0,2.0,6.0,0.0,...,16.0,0.0,3.0,1.0,12.0,0.0,520.000000,140779.191667,2,0.0
1153,950300301,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,3.0,0.0,3.0,0.0,0.0,0.0,186.000000,6779.931667,2,0.0
1154,950300350,0.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,41.000000,11099.390000,2,0.0
1155,950630012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,2.0,1.0,10.0,1.0,278.000000,20931.220000,3,1.0


On passe de 1388 lignes à 1165, alors que dès le premier merge, on devrait avoir le nombre de lignes de Q20_good, soit 1165.
On cherche à quoi c'est dû : 


- **la base par catégorie**

Dans la base Q20, trois catégories semblent se distinguer 
    - M1000 Médecine (Médecine, Neurologie, Anesthésie, Pédiatrie...)
    - M2000 Chirurgie, Ophtamologies, Gynécologie, Oto-Rhino-laryngologie
    - M3000 Autres métiers médicaux (Pharmacie, Psychiatrie, Odontologie...)

In [78]:
Q20_good_red = Q20_good.copy()
Q20_good_red['PERSO'].loc[Q20_good_red['PERSO'].astype(str).str.startswith('M1')] = "M1000"
Q20_good_red['PERSO'].loc[Q20_good_red['PERSO'].astype(str).str.startswith('M2')] = "M2000"
Q20_good_red['PERSO'].loc[Q20_good_red['PERSO'].astype(str).str.startswith('M3')] = "M3000"

C:\Users\Vinciane\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [79]:
Q20_good_red

,Numero_finess,PERSO,EFFSAL,EFFLIB
0,010008407,M1000,6.000000,0.0
1,010008407,M3000,1.666667,0.0
3,010008407,M1000,5.000000,0.0
4,010008407,M1000,1.000000,0.0
5,010008407,M1000,2.000000,0.0
...,...,...,...,...
11015,950807982,M2000,NaN,1.0
11016,950807982,M3000,NaN,6.0
11017,950807982,M3000,NaN,1.0
11018,950807982,M3000,1.000000,NaN


In [80]:
Q20_good_red[Q20_good_red['Numero_finess'] == "010008407"]

,Numero_finess,PERSO,EFFSAL,EFFLIB
0,010008407,M1000,6.000000,0.0
1,010008407,M3000,1.666667,0.0
3,010008407,M1000,5.000000,0.0
4,010008407,M1000,1.000000,0.0
5,010008407,M1000,2.000000,0.0
6,010008407,M1000,1.000000,0.0
7,010008407,M1000,3.000000,0.0
8,010008407,M1000,4.000000,0.0
9,010008407,M1000,0.000000,0.0
10,010008407,M1000,3.000000,0.0


In [81]:
Q20_good_red =  Q20_good_red.groupby(by=['Numero_finess','PERSO']).sum()

In [82]:
Q20_good_red_pivot = Q20_good_red.pivot_table(index='Numero_finess', columns='PERSO', values=['EFFSAL', 'EFFLIB'])
Q20_good_red_pivot

EFFLIB             EFFSAL                 
PERSO          M1000 M2000 M3000  M1000 M2000      M3000
Numero_finess                                           
010007300        9.0   6.0   NaN    0.0   0.0        NaN
010008407        0.0   0.0   0.0   25.0  16.0   3.666667
010009132        0.0   NaN   0.0    2.5   NaN   1.000000
010780054        9.0   0.0   2.0  130.5  27.0  21.000000
010780062        0.0   0.0   0.0   19.0   8.0   3.000000
...              ...   ...   ...    ...   ...        ...
950300277       59.0  32.0   0.0   11.0   0.0   3.000000
950300301        4.0   NaN   1.0   12.0   NaN   3.000000
950300350       11.0  12.0   1.0    1.0   0.0   1.000000
950630012        0.0   NaN   0.0   13.0   NaN   3.000000
950807982       66.0  70.0   7.0    2.0   0.0   3.000000

[1158 rows x 6 columns]

In [83]:
#On réduit Q23_good pour ne conserver que les secteurs d'activités pour les métiers et pas chaque mtier individuellement
Q23_good_red = Q23_good[Q23_good['PERSO'].isin(['N1000','N2000', 'N3000','N4000', 'N5000'])]
Q23_good_red_pivot = Q23_good_red.pivot_table(index='Numero_finess', columns='PERSO', values=['EFFPNM'])

In [84]:
# On merge les bases avec des pivots
result1_red = pd.merge(Q20_good_red_pivot, Q23_good_red_pivot, on="Numero_finess", how ="outer")

# Puis on ajoute ISA pour avoir les points ISA
result2_red = pd.merge(result1_red, ISA, on="Numero_finess" )

C:\Users\Vinciane\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [85]:
VAR_EXPL_CAT = pd.merge(result2_red, Q22_good, on=["Numero_finess"], how= "outer" )
VAR_EXPL_CAT

,Numero_finess,"(EFFLIB, M1000)","(EFFLIB, M2000)","(EFFLIB, M3000)","(EFFSAL, M1000)","(EFFSAL, M2000)","(EFFSAL, M3000)","(EFFPNM, N1000)","(EFFPNM, N2000)","(EFFPNM, N3000)","(EFFPNM, N4000)","(EFFPNM, N5000)",Points_ISA,stjr,INTTOT
0,010007300,9.0,6.0,NaN,0.0,0.0,NaN,3.000000,8.000000,NaN,NaN,NaN,3785.960000,2,NaN
1,010008407,0.0,0.0,0.0,25.0,16.0,3.666667,23.333333,110.666667,2.333333,10.333333,28.0,165054.798333,1,NaN
2,010009132,0.0,NaN,0.0,2.5,NaN,1.000000,9.500000,73.500000,2.500000,1.500000,17.5,5715.170000,1,NaN
3,010780054,9.0,0.0,2.0,130.5,27.0,21.000000,134.000000,426.333333,8.500000,109.000000,116.0,787929.068333,1,66.0
4,010780062,0.0,0.0,0.0,19.0,8.0,3.000000,62.000000,312.000000,6.000000,32.000000,61.0,188174.950000,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,950300350,11.0,12.0,1.0,1.0,0.0,1.000000,8.000000,29.000000,NaN,3.000000,1.0,11099.390000,2,NaN
1156,950630012,0.0,NaN,0.0,13.0,NaN,3.000000,28.000000,188.000000,42.000000,6.000000,14.0,20931.220000,3,1.0
1157,950807982,66.0,70.0,7.0,2.0,0.0,3.000000,72.000000,370.000000,2.000000,9.000000,6.0,177318.826667,2,NaN
1158,330780255,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0,22251.530000,2,NaN


In [86]:
VAR_EXPL_CAT.groupby(by="Numero_finess").sum()

,"(EFFLIB, M1000)","(EFFLIB, M2000)","(EFFLIB, M3000)","(EFFSAL, M1000)","(EFFSAL, M2000)","(EFFSAL, M3000)","(EFFPNM, N1000)","(EFFPNM, N2000)","(EFFPNM, N3000)","(EFFPNM, N4000)","(EFFPNM, N5000)",Points_ISA,stjr,INTTOT
Numero_finess,,,,,,,,,,,,,,
010007300,9.0,6.0,0.0,0.0,0.0,0.000000,3.000000,8.000000,0.000000,0.000000,0.0,3785.960000,2,0.0
010008407,0.0,0.0,0.0,25.0,16.0,3.666667,23.333333,110.666667,2.333333,10.333333,28.0,165054.798333,1,0.0
010009132,0.0,0.0,0.0,2.5,0.0,1.000000,9.500000,73.500000,2.500000,1.500000,17.5,5715.170000,1,0.0
010780054,9.0,0.0,2.0,130.5,27.0,21.000000,134.000000,426.333333,8.500000,109.000000,116.0,787929.068333,1,66.0
010780062,0.0,0.0,0.0,19.0,8.0,3.000000,62.000000,312.000000,6.000000,32.000000,61.0,188174.950000,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950300277,59.0,32.0,0.0,11.0,0.0,3.000000,96.000000,397.000000,2.000000,9.000000,16.0,140779.191667,2,0.0
950300301,4.0,0.0,1.0,12.0,0.0,3.000000,28.000000,150.000000,2.000000,3.000000,3.0,6779.931667,2,0.0
950300350,11.0,12.0,1.0,1.0,0.0,1.000000,8.000000,29.000000,0.000000,3.000000,1.0,11099.390000,2,0.0


En tout on a 1160 établissements, soit autant que dans Q20.

# BROUILLON EN DESSOUS

### 2) Le problème des effectifs libéraux

On cehrche à savoir s'il y a dupersonnel libéral dans notre base de données restreinte avec les FINESS des établissements MCO. L'inconvénient du personnel libéral, c'est que nous n'avons pas les **ETP, ie Equivalents Temps Plein** pour ce type de personnel, alors que nous l'avons pour tous les autres.

In [87]:
Q20_good = Q20_good[['Numero_finess','stjr','EFFSAL','EFFLIB']]
Q20_good

KeyError: "['stjr'] not in index"

In [ ]:
Q20_good_lib = Q20_good.fillna(0)
Q20_good_lib = Q20_good_lib[Q20_good_lib["EFFLIB"]!=0]
Q20_good_lib.groupby(by=['stjr', "Numero_finess"]).sum()

Au total, **533 établissements de catégories privé lucratif et privé non lucratif emploient du personnel libéral**. 
Comme nous n'avons pas les ETP pour le personnel libéral, nous allons devoir nous pencher sur **les effectifs**, pour toutes les catégories de personnel.

## Si on ne prend que les effectifs de chaque personnel dans la variable explicative

In [ ]:
EFF= pd.merge(Q20,Q22[["FI", "INTTOT"]],left_on="FI", right_on='FI',how='inner')
EFF

In [ ]:
# On ne garde pas l'information consernant le genre des salariés
Q20_EFF_SAL['EFFSALPL'] = Q20_EFF_SAL[['EFFSALPLH','EFFSALPLF']].sum(axis=1)
Q20_EFF_SAL['EFFSALPA'] = Q20_EFF_SAL[['EFFSALPAH','EFFSALPAF']].sum(axis=1)
Q20_EFF_SAL['EFFLIBPL'] = Q20_EFF_SAL[['EFFLIBPLH','EFFLIBPLF']].sum(axis=1)
Q20_EFF_SAL['EFFLIBPA'] = Q20_EFF_SAL[['EFFLIBPAH','EFFLIBPAF']].sum(axis=1)

In [ ]:
Q20_EFF_SAL = Q20_EFF_SAL.drop(['EFFSALPLF', 'EFFSALPLH', 'EFFSALPAF','EFFSALPAH', 'EFFLIB'], axis=1)
Q20_EFF_SAL = Q20_EFF_SAL[Q20_EFF_SAL['PERSO'] != "M9999"]

In [ ]:
Q20_EFF_SAL = Q20_EFF_SAL.groupby(by = ['FI', 'PERSO']).sum()